In [1]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn import svm
from keras.optimizers import SGD, Adam
from keras.optimizers import RMSprop
import pandas as pd
import numpy as np
import re
import sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', 50)
np.set_printoptions(threshold=np.inf)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

# Load in the train and test datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input//test.csv')
data=train
# Store our passenger ID for easy access
PassengerId = test['PassengerId']

# print(train.head(3))

# data
full_data = [train, test]

# Some features of my own that I have added in
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)


# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""


# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(
        ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map({'female': 0, 'male': 1}).astype(int)

    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

    # Mapping Fare
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

    # Mapping Age
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age'] = 4;


# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)
# print(train.head(3))

colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0,square=True, cmap=colormap, linecolor='white', annot=True)
# plt.show()

g = sns.pairplot(train[[u'Survived', u'Pclass', u'Sex', u'Age', u'Parch', u'Fare', u'Embarked',
       u'FamilySize', u'Title']], hue='Survived', palette = 'seismic',size=1.2,diag_kind = 'kde',diag_kws=dict(shade=True),plot_kws=dict(s=10) )
g.set(xticklabels=[])
# plt.show()


# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

x_train_tr=x_train[0:601]
x_train_te=x_train[601:891]
y_train_tr=y_train[0:601]
y_train_te=y_train[601:891]

# ニューラルネットワーク構築
model = Sequential()
model.add(Dense(20, input_dim = 11, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, nb_epoch=1000,batch_size=100, validation_split=0.2)
pred_tr=model.predict(x_train_tr)
pred_te=model.predict(x_train_te)
pred_test=model.predict(x_test)
[loss,accuracy_score] = model.evaluate(x_train_te, y_train_te)
print("loss:{0} -- accuracy:{1}".format(loss,accuracy_score))

p = [1 if i > 0.5 else 0 for i in pred_te]
print('Accuracy at NN:n',sum(p == y_train_te) / 290.0)

# svm
classifier = svm.SVC(C=1.0, gamma=0.01)
classifier.fit(pred_tr, y_train_tr)
predicted = classifier.predict(pred_te)
print('Accuracy at NN+svm1:n', metrics.accuracy_score(y_train_te, predicted))

param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}]

clf = GridSearchCV(SVC(C=1), param_grid, cv=5, n_jobs=-1)  # n_jobs: 並列計算を行う（-1 とすれば使用PCで可能な最適数の並列処理を行う）
clf.fit(x_train, y_train)
# print(clf.best_estimator_)  # 最適なパラメータを表示

pred=clf.predict(x_test)
# print(classification_report(y_train_te, pred))  # クラスタリング結果を表示
# print(confusion_matrix(y_train_te, pred))  # クラスタリング結果を表示
# print('Accuracy at NN+svm2:n', metrics.accuracy_score(y_train_te, pred))

# output_df
test_pred=model.predict(x_test)
test_pred = [1 if i > 0.5 else 0 for i in test_pred]

output_df = pd.DataFrame(test_pred, columns=['Survived'])
output_df['PassengerId'] = PassengerId
output_df = output_df.ix[:, ['PassengerId', 'Survived']]
output_df.to_csv('../submission/test_ans_nn.csv', index=False, encoding='utf-8')

output_df = pd.DataFrame(pred, columns=['Survived'])
output_df['PassengerId'] = PassengerId
output_df = output_df.ix[:, ['PassengerId', 'Survived']]
output_df.to_csv('../submission/test_ans_svm.csv', index=False, encoding='utf-8')

Using TensorFlow backend.


Train on 712 samples, validate on 179 samples
Epoch 1/1000
712/712 [==============================] - 0s 402us/step - loss: 1.8118 - accuracy: 0.6096 - val_loss: 0.9741 - val_accuracy: 0.6425
Epoch 2/1000
712/712 [==============================] - 0s 13us/step - loss: 0.8169 - accuracy: 0.5941 - val_loss: 0.7148 - val_accuracy: 0.5307
Epoch 3/1000
712/712 [==============================] - 0s 10us/step - loss: 0.7568 - accuracy: 0.4312 - val_loss: 0.7043 - val_accuracy: 0.5363
Epoch 4/1000
712/712 [==============================] - 0s 11us/step - loss: 0.6573 - accuracy: 0.6250 - val_loss: 0.6098 - val_accuracy: 0.6480
Epoch 5/1000
712/712 [==============================] - 0s 10us/step - loss: 0.6452 - accuracy: 0.6348 - val_loss: 0.5929 - val_accuracy: 0.6704
Epoch 6/1000
712/712 [==============================] - 0s 13us/step - loss: 0.6170 - accuracy: 0.6685 - val_loss: 0.5838 - val_accuracy: 0.7598
Epoch 7/1000
712/712 [==============================] - 0s 11us/step - loss: 0.5986

Epoch 57/1000
712/712 [==============================] - 0s 11us/step - loss: 0.4499 - accuracy: 0.8146 - val_loss: 0.3834 - val_accuracy: 0.8380
Epoch 58/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4593 - accuracy: 0.7865 - val_loss: 0.3725 - val_accuracy: 0.8436
Epoch 59/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4480 - accuracy: 0.8118 - val_loss: 0.3902 - val_accuracy: 0.8436
Epoch 60/1000
712/712 [==============================] - 0s 8us/step - loss: 0.4330 - accuracy: 0.8160 - val_loss: 0.3729 - val_accuracy: 0.8492
Epoch 61/1000
712/712 [==============================] - 0s 11us/step - loss: 0.4436 - accuracy: 0.8118 - val_loss: 0.3850 - val_accuracy: 0.8380
Epoch 62/1000
712/712 [==============================] - ETA: 0s - loss: 0.5095 - accuracy: 0.78 - 0s 10us/step - loss: 0.4401 - accuracy: 0.8188 - val_loss: 0.3738 - val_accuracy: 0.8659
Epoch 63/1000
712/712 [==============================] - 0s 10us/step - loss: 0.440

712/712 [==============================] - 0s 10us/step - loss: 0.4276 - accuracy: 0.8258 - val_loss: 0.3651 - val_accuracy: 0.8436
Epoch 113/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4241 - accuracy: 0.8272 - val_loss: 0.3564 - val_accuracy: 0.8436
Epoch 114/1000
712/712 [==============================] - 0s 11us/step - loss: 0.4342 - accuracy: 0.8174 - val_loss: 0.3605 - val_accuracy: 0.8436
Epoch 115/1000
712/712 [==============================] - 0s 8us/step - loss: 0.4252 - accuracy: 0.8258 - val_loss: 0.3607 - val_accuracy: 0.8380
Epoch 116/1000
712/712 [==============================] - 0s 11us/step - loss: 0.4240 - accuracy: 0.8272 - val_loss: 0.3598 - val_accuracy: 0.8380
Epoch 117/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4255 - accuracy: 0.8216 - val_loss: 0.3580 - val_accuracy: 0.8380
Epoch 118/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4245 - accuracy: 0.8244 - val_loss: 0.3592 - val_accu

712/712 [==============================] - 0s 11us/step - loss: 0.4160 - accuracy: 0.8329 - val_loss: 0.3501 - val_accuracy: 0.8492
Epoch 168/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4168 - accuracy: 0.8315 - val_loss: 0.3481 - val_accuracy: 0.8492
Epoch 169/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4170 - accuracy: 0.8301 - val_loss: 0.3514 - val_accuracy: 0.8492
Epoch 170/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4178 - accuracy: 0.8287 - val_loss: 0.3505 - val_accuracy: 0.8492
Epoch 171/1000
712/712 [==============================] - 0s 11us/step - loss: 0.4167 - accuracy: 0.8315 - val_loss: 0.3459 - val_accuracy: 0.8492
Epoch 172/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4258 - accuracy: 0.8216 - val_loss: 0.3454 - val_accuracy: 0.8492
Epoch 173/1000
712/712 [==============================] - 0s 11us/step - loss: 0.4154 - accuracy: 0.8287 - val_loss: 0.3505 - val_acc

...

Epoch 777/1000
712/712 [==============================] - 0s 10us/step - loss: 0.4068 - accuracy: 0.8385 - val_loss: 0.3824 - val_accuracy: 0.8492
Epoch 778/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3890 - accuracy: 0.8399 - val_loss: 0.3669 - val_accuracy: 0.8492
Epoch 779/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3872 - accuracy: 0.8315 - val_loss: 0.3783 - val_accuracy: 0.8492
Epoch 780/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3848 - accuracy: 0.8413 - val_loss: 0.3704 - val_accuracy: 0.8603
Epoch 781/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3844 - accuracy: 0.8399 - val_loss: 0.3772 - val_accuracy: 0.8436
Epoch 782/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3870 - accuracy: 0.8343 - val_loss: 0.3675 - val_accuracy: 0.8547
Epoch 783/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3956 - accuracy: 0.8272 - val_loss: 0

Epoch 833/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3862 - accuracy: 0.8329 - val_loss: 0.3820 - val_accuracy: 0.8547
Epoch 834/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3841 - accuracy: 0.8315 - val_loss: 0.3752 - val_accuracy: 0.8603
Epoch 835/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3801 - accuracy: 0.8427 - val_loss: 0.3939 - val_accuracy: 0.8436
Epoch 836/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3833 - accuracy: 0.8455 - val_loss: 0.3747 - val_accuracy: 0.8492
Epoch 837/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3787 - accuracy: 0.8413 - val_loss: 0.3825 - val_accuracy: 0.8436
Epoch 838/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3798 - accuracy: 0.8427 - val_loss: 0.3790 - val_accuracy: 0.8492
Epoch 839/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3804 - accuracy: 0.8371 - val_loss: 0

Epoch 889/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3806 - accuracy: 0.8441 - val_loss: 0.3567 - val_accuracy: 0.8603
Epoch 890/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3822 - accuracy: 0.8315 - val_loss: 0.3694 - val_accuracy: 0.8436
Epoch 891/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3789 - accuracy: 0.8441 - val_loss: 0.3833 - val_accuracy: 0.8436
Epoch 892/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3858 - accuracy: 0.8343 - val_loss: 0.3771 - val_accuracy: 0.8659
Epoch 893/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3843 - accuracy: 0.8315 - val_loss: 0.3789 - val_accuracy: 0.8547
Epoch 894/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3762 - accuracy: 0.8455 - val_loss: 0.3803 - val_accuracy: 0.8492
Epoch 895/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3763 - accuracy: 0.8413 - val_loss: 0

Epoch 945/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3808 - accuracy: 0.8399 - val_loss: 0.3671 - val_accuracy: 0.8603
Epoch 946/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3743 - accuracy: 0.8469 - val_loss: 0.3752 - val_accuracy: 0.8547
Epoch 947/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3729 - accuracy: 0.8455 - val_loss: 0.3614 - val_accuracy: 0.8492
Epoch 948/1000
712/712 [==============================] - 0s 13us/step - loss: 0.3763 - accuracy: 0.8371 - val_loss: 0.3702 - val_accuracy: 0.8547
Epoch 949/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3766 - accuracy: 0.8399 - val_loss: 0.3686 - val_accuracy: 0.8603
Epoch 950/1000
712/712 [==============================] - 0s 10us/step - loss: 0.3732 - accuracy: 0.8497 - val_loss: 0.3804 - val_accuracy: 0.8492
Epoch 951/1000
712/712 [==============================] - 0s 11us/step - loss: 0.3785 - accuracy: 0.8357 - val_loss: 0

290/290 [==============================] - 0s 10us/step
loss:0.36159699292018493 -- accuracy:0.8448275923728943
Accuracy at NN:n 0.8448275862068966
Accuracy at NN+svm1:n 0.8310344827586207
